In [74]:
import pandas as pd

df_mnq = pd.read_csv("../../data-461318792@60.csv")
df_mnq = df_mnq[["date", "close"]]
df_mym = pd.read_csv("../../data-477836934@60.csv")
df_mym = df_mym[["date", "close"]]

In [75]:

df = pd.merge(df_mnq, df_mym, on="date", suffixes=("_mnq", "_mym"))
df.index = pd.to_datetime(df["date"])
df.drop(columns=["date"], inplace=True)
df

,close_mnq,close_mym
date,,
2022-03-04 15:00:00,13826.50,33549.0
2022-03-04 15:01:00,13823.50,33556.0
2022-03-04 15:02:00,13820.00,33561.0
2022-03-04 15:03:00,13818.50,33561.0
2022-03-04 15:04:00,13817.75,33560.0
...,...,...
2022-03-07 15:55:00,13274.75,32695.0
2022-03-07 15:56:00,13277.00,32697.0
2022-03-07 15:57:00,13278.50,32700.0


In [76]:
import numpy as np
import talib
from talib import MA_Type

df["px_diff"] = np.log(df["close_mym"].divide(df["close_mnq"]))
upper, mid, lower = talib.BBANDS(df["px_diff"], timeperiod=10, nbdevup=2, nbdevdn=2, matype=MA_Type.SMA)
df["px_diff_upper"] = upper
df["px_diff_mid"] = mid
df["px_diff_lower"] = lower
df

,close_mnq,close_mym,px_diff,px_diff_upper,px_diff_mid,px_diff_lower
date,,,,,,
2022-03-04 15:00:00,13826.50,33549.0,0.886420,NaN,NaN,NaN
2022-03-04 15:01:00,13823.50,33556.0,0.886846,NaN,NaN,NaN
2022-03-04 15:02:00,13820.00,33561.0,0.887248,NaN,NaN,NaN
2022-03-04 15:03:00,13818.50,33561.0,0.887356,NaN,NaN,NaN
2022-03-04 15:04:00,13817.75,33560.0,0.887381,NaN,NaN,NaN
...,...,...,...,...,...,...
2022-03-07 15:55:00,13274.75,32695.0,0.901358,0.901497,0.901214,0.900930
2022-03-07 15:56:00,13277.00,32697.0,0.901250,0.901461,0.901200,0.900939
2022-03-07 15:57:00,13278.50,32700.0,0.901229,0.901465,0.901204,0.900943


In [77]:
df_selected = df.between_time("02:30", "06:30")
df_selected

,close_mnq,close_mym,px_diff,px_diff_upper,px_diff_mid,px_diff_lower
date,,,,,,
2022-03-07 02:30:00,13562.50,32940.0,0.887379,0.889047,0.888203,0.887360
2022-03-07 02:31:00,13552.75,32931.0,0.887825,0.888968,0.888131,0.887294
2022-03-07 02:32:00,13554.00,32936.0,0.887885,0.888873,0.888068,0.887262
2022-03-07 02:33:00,13572.00,32962.0,0.887347,0.888820,0.887956,0.887093
2022-03-07 02:34:00,13578.75,32979.0,0.887365,0.888608,0.887826,0.887043
...,...,...,...,...,...,...
2022-03-07 06:26:00,13731.75,33325.0,0.886597,0.887157,0.886756,0.886355
2022-03-07 06:27:00,13728.25,33326.0,0.886882,0.887179,0.886771,0.886364
2022-03-07 06:28:00,13749.50,33377.0,0.886865,0.887156,0.886762,0.886368


In [78]:
from datetime import time
from pandas import DataFrame

executions = []
current_pnl = 0

MNQ_LOT = 2
MYM_LOT = 6

for dt, row in df_selected.iterrows():
    last = executions[-1] if executions else None

    # Exit - no cross-day position
    if dt.time() == time(6, 30):
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }

    # Entry - out of band
    if not last or last["position"] == "close":
        if row["px_diff"] > row["px_diff_upper"]:
            executions.append({
                "buy": "MNQ",
                "sell": "MYM",
                "entry_t": dt,
                "entry_mnq": row["close_mnq"],
                "entry_mym": row["close_mym"],
                "exit_mnq": None,
                "exit_mym": None,
                "exit_t": None,
                "position": "open",
                "pnl_highest": 0,
                "pnl_lowest": 0,
            })

        if row["px_diff"] < row["px_diff_lower"]:
            executions.append({
                "buy": "MYM",
                "sell": "MNQ",
                "entry_t": dt,
                "entry_mnq": row["close_mnq"],
                "entry_mym": row["close_mym"],
                "exit_mnq": None,
                "exit_mym": None,
                "exit_t": None,
                "position": "open",
                "pnl_highest": 0,
                "pnl_lowest": 0,
            })

        continue
    # Take profit - back to MID
    elif (
            (last["buy"] == "MNQ" and row["px_diff"] < row["px_diff_mid"]) or
            (last["buy"] == "MYM" and row["px_diff"] > row["px_diff_mid"])
    ):
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }
        current_pnl = 0

    # Record PnL
    if last["position"] == "open":
        pnl_mnq = (row["close_mnq"] - last["entry_mnq"]) * (1 if last["buy"] == "MNQ" else -1) * MNQ_LOT * 2
        pnl_mym = (row["close_mym"] - last["entry_mym"]) * (1 if last["buy"] == "MYM" else -1) * MYM_LOT * 0.5

        current_pnl = pnl_mnq + pnl_mym

        executions[-1] |= {
            "pnl_highest": max(last["pnl_highest"], current_pnl),
            "pnl_lowest": min(last["pnl_lowest"], current_pnl),
        }

    # Take profit - lock profit
    if current_pnl < 10 and last["pnl_highest"] > 40:
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }

    # Stop loss - force close
    if current_pnl < -50:
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }

df_exec = DataFrame(executions)
df_exec["hold_period"] = df_exec["exit_t"] - df_exec["entry_t"]
df_exec["px_side_mnq"] = (df_exec["exit_mnq"] - df_exec["entry_mnq"]) * np.where(df_exec["buy"] == "MNQ", 1, -1)
df_exec["px_side_mym"] = (df_exec["exit_mym"] - df_exec["entry_mym"]) * np.where(df_exec["buy"] == "MYM", 1, -1)
df_exec["pnl_mnq"] = df_exec["px_side_mnq"] * MNQ_LOT * 2
df_exec["pnl_mym"] = df_exec["px_side_mym"] * MYM_LOT * 0.5
df_exec["pnl_single"] = df_exec["pnl_mnq"] + df_exec["pnl_mym"] - 0.52 * (MNQ_LOT + MYM_LOT)
df_exec["pnl_cum"] = df_exec["pnl_single"].cumsum()
df_exec

,buy,sell,entry_t,entry_mnq,entry_mym,exit_mnq,exit_mym,exit_t,position,pnl_highest,pnl_lowest,hold_period,px_side_mnq,px_side_mym,pnl_mnq,pnl_mym,pnl_single,pnl_cum
0,MYM,MNQ,2022-03-07 02:39:00,13606.00,33027.0,13593.75,33020.0,2022-03-07 02:42:00,close,3.0,-16.0,0 days 00:03:00,12.25,-7.0,49.0,-21.0,23.84,23.84
1,MNQ,MYM,2022-03-07 02:59:00,13601.50,33094.0,13616.00,33133.0,2022-03-07 03:02:00,close,0.0,-59.0,0 days 00:03:00,14.50,-39.0,58.0,-117.0,-63.16,-39.32
2,MYM,MNQ,2022-03-07 03:19:00,13627.50,33130.0,13620.50,33114.0,2022-03-07 03:25:00,close,13.0,-35.0,0 days 00:06:00,7.00,-16.0,28.0,-48.0,-24.16,-63.48
3,MNQ,MYM,2022-03-07 03:28:00,13625.25,33153.0,13602.00,33092.0,2022-03-07 03:37:00,close,91.0,0.0,0 days 00:09:00,-23.25,61.0,-93.0,183.0,85.84,22.36
4,MNQ,MYM,2022-03-07 03:43:00,13604.00,33118.0,13598.25,33090.0,2022-03-07 03:44:00,close,0.0,0.0,0 days 00:01:00,-5.75,28.0,-23.0,84.0,56.84,79.20
5,MYM,MNQ,2022-03-07 04:05:00,13610.50,33102.0,13614.25,33110.0,2022-03-07 04:11:00,close,4.0,-33.0,0 days 00:06:00,-3.75,8.0,-15.0,24.0,4.84,84.04
6,MYM,MNQ,2022-03-07 04:40:00,13606.75,33055.0,13617.50,33093.0,2022-03-07 04:45:00,close,31.0,0.0,0 days 00:05:00,-10.75,38.0,-43.0,114.0,66.84,150.88
7,MNQ,MYM,2022-03-07 04:56:00,13623.75,33126.0,13616.00,33093.0,2022-03-07 04:58:00,close,69.0,0.0,0 days 00:02:00,-7.75,33.0,-31.0,99.0,63.84,214.72
8,MYM,MNQ,2022-03-07 05:29:00,13604.75,33069.0,13616.50,33084.0,2022-03-07 05:40:00,close,33.0,-48.0,0 days 00:11:00,-11.75,15.0,-47.0,45.0,-6.16,208.56
9,MNQ,MYM,2022-03-07 05:41:00,13618.75,33097.0,13630.00,33112.0,2022-03-07 05:45:00,close,0.0,-10.0,0 days 00:04:00,11.25,-15.0,45.0,-45.0,-4.16,204.40


In [79]:
import plotly.express as px

fig = px.line(df_exec, x=df_exec.index, y="pnl_cum")
fig.show()